<a href="https://colab.research.google.com/github/CMallart/ateliers-NN/blob/main/BigCNNMushroomClassif.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import datetime, os
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from sklearn.metrics import balanced_accuracy_score, confusion_matrix


def build_CNN(input_dim, output_dim, lr=0.001):
    model = Sequential()

    model.add(Conv2D(32, (3, 3), input_shape=input_dim, activation="relu"))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(rate=0.1))

    model.add(Conv2D(64, (3, 3), input_shape=input_dim, activation="relu"))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(rate=0.1))

    model.add(Conv2D(64, (3, 3), input_shape=input_dim, activation="relu"))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(rate=0.1))

    model.add(Flatten())
    model.add(Dense(64, activation="relu"))
    model.add(Dropout(rate=0.1))

    model.add(Dense(output_dim))

    model.compile(loss=CategoricalCrossentropy(from_logits=True), optimizer='rmsprop', metrics=['accuracy'])

    return model


def build_CNN_from_pretrained(file_name, output_dim, lr=0.001):
    model = load_model(file_name)
    model.pop()

    model.add(Dense(output_dim, name='dense_2'))

    model.compile(loss=CategoricalCrossentropy(from_logits=True), optimizer='adam', metrics=['accuracy'])
    return model

In [3]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
#Q2: Ajouter l'attribut rescale aux générateurs
train_generator = ImageDataGenerator(rescale=1/255,
                                     rotation_range=20,
                                     width_shift_range=0.2,
                                     height_shift_range=0.2,
                                     horizontal_flip=True,
                                     zoom_range=0.1)
valid_generator = ImageDataGenerator(rescale=1/255)

epochs = 50
batch_size = 64

#Q2: Ajouter les attributs "target_size" et "batch_size"
train_flow = train_generator.flow_from_directory("data/train", class_mode='categorical',
        target_size=(32, 32), batch_size=batch_size)
valid_flow = train_generator.flow_from_directory("data/valid", class_mode='categorical',
        target_size=(32, 32), batch_size=batch_size)

from_scratch = False
if from_scratch:
    # Q2: compléter l'appel à la fonction build_CNN()
    cnn_model = build_CNN((32, 32, 3), 2)
    label = 'from_scratch'
else:
    cnn_model = build_CNN_from_pretrained('pretrained_model.h5', 2)
    label = 'from_pretrained'
cnn_model.summary()

#Q2: compléter l'appel aux fonctions fit() et evaluate()
tensorboard_callback = TensorBoard(log_dir='logs/' + label, histogram_freq=int(epochs/10))

cnn_model.fit(
        train_flow,
        epochs=epochs,
        validation_data=valid_flow,
        callbacks=[tensorboard_callback])

cnn_model.evaluate(valid_flow, verbose=2)

